In [1]:
import os
from string import Template

In [5]:
SLURM_DIR = '../SLURMS/Figure4_From_Scratch/'
LOG_DIR = SLURM_DIR+'LOGS/'
os.makedirs(SLURM_DIR)
os.makedirs(LOG_DIR)

OSError: [Errno 17] File exists: '../SLURMS/Figure4_From_Scratch/'

In [6]:
SLURM_TEMPLATE = Template('''#!/bin/bash
#SBATCH -p cox
#SBATCH -n 1 # Number of cores
#SBATCH -N 1 # Ensure that all cores are on one machine
#SBATCH --gres=gpu
#SBATCH --array=9-12
#SBATCH --mem=$memory
#SBATCH -t 3-00:00
#SBATCH --mail-type=ALL
#SBATCH --mail-user=haehn@seas.harvard.edu
#SBATCH -o /n/home05/haehn/Projects/CP/SLURMS/Figure4_From_Scratch/LOGS/%A_%a.out
#SBATCH -e /n/home05/haehn/Projects/CP/SLURMS/Figure4_From_Scratch/LOGS/%A_%a.err

# Print this sub-job's task ID
echo "My SLURM_ARRAY_TASK_ID: " $SLURM_ARRAY_TASK_ID

source new-modules.sh

module load Anaconda/5.0.1-fasrc01

cd /n/home05/haehn/Projects/CP/EXP/

source activate CP

python run_position_length_from_scratch.py $experiment $classifier $noise $SLURM_ARRAY_TASK_ID

# end
exit 0;
''')

In [7]:
Experiments = ['C.Figure4.data_to_type1', 'C.Figure4.data_to_type2',\
              'C.Figure4.data_to_type3', 'C.Figure4.data_to_type4', 'C.Figure4.data_to_type5', 'C.Figure4.multi']

Classifiers = {
#   'MLP': '8000',
#   'LeNet': '8000',
  'VGG19': '20000',
  'XCEPTION': '32000'
}

Noise = ['True']#, 'False']

for e in Experiments:
  for c in Classifiers:
    m = Classifiers[c]
    for n in Noise:

      new_slurm = SLURM_TEMPLATE.substitute(experiment=e, memory=m, classifier=c, noise=n, SLURM_ARRAY_TASK_ID='$SLURM_ARRAY_TASK_ID')
      with open(os.path.join(SLURM_DIR, e+'_'+c+'_'+n+'.sbatch'), 'w') as f:
        f.write(new_slurm)


In [ ]:
for f in *_True.sbatch; do sbatch $f; done